In [1]:
#Python API for ltr11 is required
from ltr11 import *
import threading
import concurrent.futures
import time
import pprint
import queue
import csv
import serial
import os
import numpy as np
import re

In [2]:
def get_radar_data (event, queue) :
# Accesses IQ data on BGT60LTR11 radar board and passes into a queue
    flag = True
    try:
        with BGT60LTR11() as ltr11 :
            ltr11.set_configuration(mode = 0)  # 0 = continuous, 1 = pulse
            config = ltr11.get_configuration( )
            print("Configuration:")
            pprint.pprint(ltr11.get_configuration( ))
            ltr11.start_data_acquisition( )

            while not event.is_set( ) :
                ifi ,ifq = ltr11.get_data(1000) # 2 kHz / 2
                if flag:
                    radar_time=time.time()
                    print('first radar data at {}'.format(radar_time))
                    flag = False
                data = zip(ifi.tolist(),ifq.tolist())
                for row in data :
                    queue.put(row)
            print('radar stopped {}'.format(time.time()))
            ltr11.stop_data_acquisition()
            print('radar acquisition stopped')
    except:
        print('Exception: Is the radar board connected?')
        event.set()

In [3]:
def get_ecg_data(event, serial, queue):
    flag = True
    try:
        serial.close()
        serial.open()
        while not event.is_set():
            data = []
            for i in range(250):    # 500 Hz / 2
                line = serial.readline()
                #there are four values in the line_sequence = [-ECG1 -ECG2 -ECG3 -temperature sensor]
                line_sequence = line.decode("utf-8")
                #print('This is the line_sequence from the serial port:{}'.format(line_sequence))
                #use match function to record the ECG1
                #match = re.findall(r'\-?\d+', line_sequence)
                #data += [int(match[1])]
                data += [int(line_sequence)]
                #print('This is the ECG value selected from the serial port:{}'.format(int(match[1])))
                if flag:
                    ECG_time=time.time()
                    print('first ecg data at {}'.format(ECG_time))
                    flag = False
            for row in zip(data):
                queue.put(row)
        print('ecg stopped {}/n'.format(time.time()))
        serial.close()
        print('serial port closed')
        queue.put([])  #Prevent queue.get()  call getting stuck
    except:
        print('ecg error')
        serial.close()
        print('serial port closed')
        queue.put([])  #Prevent queue.get()  call getting stuck
        event.set()

In [4]:
def write_data(event, queue, f):
    try:
        wr = csv.writer(f)
        while not (event.is_set() and queue.empty()):
            data = queue.get()
            wr.writerow(data)
        print('Closing {}'.format(os.path.basename(f.name)))
        f.close()
        print('{} closed'.format(os.path.basename(f.name)))
    except:
        print('{} writer error'.format(f))

In [5]:
def init():
    print('init')
    ax.set_xlim(0, 2*np.pi)
    ax.set_ylim(-1, 1)
    return ln,

In [6]:
def update(frame):
    print('update')
    xdata.append(frame)
    ydata.append(np.sin(frame))
    ln.set_data(xdata, ydata)
    return ln,

In [7]:
if __name__ == "__main__":
    sample_dur = 35  # Sample duration in seconds
    start = time.time()
    timestr = time.strftime("%Y%m%d-%H%M%S")
    fname = "{}-{}.csv".format(timestr,'radar')
    #fpath = "../logs/{}".format(fname)
    fpath = "{}".format(fname)
    f1 = open(fpath, 'w', newline="")
    fname = "{}-{}.csv".format(timestr,'ecg')
    #fpath = "../logs/{}".format(fname)
    fpath = "{}".format(fname)
    f2 = open(fpath, 'w', newline="")
    
    event = threading.Event()
    radar_pipeline = queue.Queue()
    ecg_pipeline = queue.Queue()
    
    # Set up serial linstener
    #serial_port = 'COM5'
    #the serial_port used in my laptop is actually COM3 instead of COM5
    serial_port = 'COM3'
    baud_rate = 921600;
    ser = serial.Serial(serial_port, baud_rate)
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=None) as executor:
        print('starting threads')
        radar_thread = executor.submit(get_radar_data, event, radar_pipeline)
        ecg_thread = executor.submit(get_ecg_data, event, ser, ecg_pipeline)
        radar_writer = executor.submit(write_data, event, radar_pipeline, f1)
        ecg_writer = executor.submit(write_data, event, ecg_pipeline, f2)
        try:
            while not (radar_thread.done() and ecg_thread.done() and radar_writer.done() and ecg_writer.done()):
                if time.time() - start >= sample_dur:
                    event.set()
                    time.sleep(0.1)
                else:
                    time.sleep(0.5)
                    
        except KeyboardInterrupt:
            print('Keyboard Interrupt')
            event.set()
       
            
        print('Radar result: {}'.format(radar_thread.result()))
        print('ECG result: {}'.format(ecg_thread.result()))
        print('Radar writer result:{}'.format(radar_writer.result()))
        print('ECG writer result:{}'.format(ecg_writer.result()))
        print('elapsed time = {}'.format(time.time() - start))
        

starting threads
Configuration:
{'adc': 0,
 'detection_threshold': 0,
 'hold_time': 4,
 'mode': 0,
 'pulse_repetition': 1,
 'pulse_width': 0,
 'rf_center_freq': 1,
 'rx_if_gain': 8,
 'sampling_frequency': 2000,
 'tx_power_level': 7}
first radar data at 1684336520.0666323
first ecg data at 1684336521.3712604
ecg stopped 1684336555.035552/nClosing 20230517-161519-ecg.csv

20230517-161519-ecg.csv closedserial port closed

radar stopped 1684336555.06426
radar acquisition stopped
Closing 20230517-161519-radar.csv
20230517-161519-radar.csv closed
Radar result: None
ECG result: None
Radar writer result:None
ECG writer result:None
elapsed time = 35.6523003578186


In [8]:
5.192809-3.8707228

1.3220861999999998